In [60]:
import Pkg
using Flux
using CSV, DataFrames
using Flux: crossentropy, normalise, onecold, onehot, Params, gradient
using Statistics: mean
using Flux.Tracker, Statistics, DelimitedFiles
using Flux.Tracker: Params, gradient, update!
using DelimitedFiles, Statistics
using Flux: gpu
using LinearAlgebra

In [52]:
data = DataFrame(CSV.File("C:\\Users\\tigra\\Documents\\NBA 4920\\Project_train.csv"))
labels = data[:,4]
features = data[:,1:3]
features = hcat(features, data[:,5])
#labels = onehot(:1, labels)

,customer,income,mailer,x1
,Int64,Int64,Int64,Float64
1,0,39,0,0.0
2,0,39,0,0.693147
3,0,39,1,0.693147
4,0,39,1,0.693147
5,0,39,0,1.09861
6,0,39,1,1.38629
7,0,39,0,1.38629
8,0,39,0,1.60944
9,0,39,0,1.79176


In [68]:
features = convert(Matrix, features)

# Subract mean, divide by std dev for normed mean of 0 and std dev of 1.
normed_features = normalise(features)
#labels
normed_features = transpose(normed_features)
#klasses = sort(unique(labels))
#onehot_labels = onehotbatch(labels, klasses)


# Split into training and test sets, 2/3 for training, 1/3 for test.

x_train = normed_features[1:4, 1:8000]
y_train = labels[1:8000]
#y_train = transpose(y_train)

x_test = normed_features[1:4, 8001:10000]
y_test = labels[8001:10000]


2000-element Array{Int64,1}:
 0
 0
 0
 0
 0
 1
 1
 0
 0
 1
 0
 0
 0
 ⋮
 0
 0
 0
 0
 1
 0
 1
 1
 0
 0
 0
 0

In [79]:
W = param(randn(1,4)/10)
b = param([0.])

predict(x) = W*x .+ b
meansquarederror(ŷ, y) = sum((ŷ .- y).^2)/size(y, 2)
loss(x, y,i) = meansquarederror(predict(x[i]), y[i])

η = 0.1
θ = Params([W, b])

for i = 1:50
  g = gradient(() -> loss(x_train, y_train,i), θ)
  for x in θ
    update!(x, -g[x]*η)
  end
  #@show loss(x_train, y_train)
end

# Predict the RMSE on the test set
err = meansquarederror(predict(x_test),y_test)

1.5653896868254775e6 (tracked)

In [89]:
data1 = DataFrame(CSV.File("C:\\Users\\tigra\\Documents\\NBA 4920\\Project_test.csv"))
features1 = data1[:,1:3]
features1 = hcat(features1, data1[:,5])
features1 = convert(Matrix, features1)

# Subract mean, divide by std dev for normed mean of 0 and std dev of 1.
normed_features1 = normalise(features1)
#labels
normed_features1 = transpose(normed_features1)
#klasses = sort(unique(labels))
#onehot_labels = onehotbatch(labels, klasses)


# Split into training and test sets, 2/3 for training, 1/3 for test.

x_train = normed_features1[1:4, 1:1250]
y_test = predict(x_train)


Tracked 1×1250 Array{Float64,2}:
 2.21416  2.21355  2.21294  2.21233  …  -0.440888  -0.459714  -0.460325